In [1]:
# %pip install ultralytics
# %pip install pycocotools requests click
# !git clone https://github.com/tw-yshuang/coco2yolo.git
# !chmod +x ./coco2yolo/coco2yolo
# %pip install -U ultralytics
# %pip install torch torchvision torchaudio

In [2]:
from ultralytics import YOLO
import json
import os
import shutil
import torch
from torch import __version__ as torch_ver
import os
import json
from tqdm import tqdm
import shutil
import random
import cv2
from ultralytics.engine.results import Results
from matplotlib import pyplot as plt

In [3]:
torch.cuda.is_available(), torch.cuda.device_count()

(True, 1)

In [4]:
TORCH_VERSION = ".".join(torch_ver.split(".")[:2])
CUDA_VERSION = torch_ver.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("gpu:", torch.cuda.is_available())

torch:  2.2 ; cuda:  cu121
gpu: True


In [5]:
%ls "./yolov9c-50k_iter-output/train5/weights/"

best.pt  epoch10.pt  last.pt


In [6]:
MODEL_WEIGHTS = "./yolov9c-50k_iter-output/train5/weights/best.pt"
MODEL = "yolov9c"
OUTPUT_DIR = "./" + MODEL + "-50k_iter-output"
DATASET_PATH = "./DATASET_BIG_YOLO"
YAML_PATH = f"{DATASET_PATH}/AGAR.yaml"
TRAINING_LISTS_PATH = "./DATASET_BIG/training_lists/"
CATEGORIES = ["S.aureus",
              "B.subtilis",
              "P.aeruginosa",
              "E.coli",
              "C.albicans"]  # исключили классы Defect и Contamination
CATEGORIES_DICT = {0: "S.aureus",
                   1: "B.subtilis",
                   2: "P.aeruginosa",
                   3: "E.coli",
                   4: "C.albicans"}  # исключили классы Defect и Contamination

print(MODEL)
print(OUTPUT_DIR)

yolov9c
./yolov9c-50k_iter-output


In [7]:
def get_ids_by_train_list(training_list_path: str) -> list[int]:
    with open(training_list_path, 'r') as f:
        data = f.read()
    j = json.loads(data)
    str_ids = [num for num in j if num != 'annotations.json']
    ids = [int(num) for num in str_ids]
    return ids


def copy_photos_and_txt(photo_ids, source_dir, destination_dir):
    """
    Moves photos and corresponding text files with specified IDs.

    Args:
        photo_ids: A list of integers representing photo IDs.
        source_dir: The directory containing the photos and text files.
        destination_dir: The directory to move the files to.
    """
    for photo_id in tqdm(photo_ids, desc=f"Copying {len(photo_ids)} jpgs and txts from {source_dir} to {destination_dir} dir"):
        photo_filename = f"{photo_id}.jpg"
        text_filename = f"{photo_id}.txt"

        photo_source_path = os.path.join(source_dir, photo_filename)
        text_source_path = os.path.join(source_dir, text_filename)

        photo_destination_path = os.path.join(destination_dir, photo_filename)
        text_destination_path = os.path.join(destination_dir, text_filename)

        for source_path, destination_path in [(photo_source_path, photo_destination_path), (text_source_path, text_destination_path)]:
            if os.path.exists(source_path):
                shutil.copy(source_path, destination_path)
            else:
                print(
                    f"Warning: File with ID {photo_id} ({os.path.basename(source_path)}) not found in source directory {source_dir}.")


def convert_bbox_coco2yolo(img_width, img_height, bbox):
    """
    Convert bounding box from COCO  format to YOLO format

    Parameters
    ----------
    img_width : int
        width of image
    img_height : int
        height of image
    bbox : list[int]
        bounding box annotation in COCO format: 
        [top left x position, top left y position, width, height]

    Returns
    -------
    list[float]
        bounding box annotation in YOLO format: 
        [x_center_rel, y_center_rel, width_rel, height_rel]
    """

    # YOLO bounding box format: [x_center, y_center, width, height]
    # (float values relative to width and height of image)
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]


def make_folders(path="output"):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    return path


def convert_coco_json_to_yolo_txt(output_path, json_file):
    path = make_folders(output_path)

    with open(json_file) as f:
        json_data = json.load(f)

    label_file = os.path.join(output_path, "labels")
    with open(label_file, "w") as f:
        for category in tqdm(json_data["categories"], desc="Categories"):
            category_name = category["name"]
            f.write(f"{category_name}\n")

    for image in tqdm(json_data["images"], desc="Annotation txt for each iamge"):
        img_id = image["id"]
        img_name = image["file_name"]
        img_width = image["width"]
        img_height = image["height"]

        anno_in_image = [
            anno for anno in json_data["annotations"] if anno["image_id"] == img_id]
        anno_txt = os.path.join(output_path, img_name.split(".")[0] + ".txt")
        with open(anno_txt, "w") as f:
            for anno in anno_in_image:
                category = anno["category_id"]
                bbox_COCO = anno["bbox"]
                x, y, w, h = convert_bbox_coco2yolo(
                    img_width, img_height, bbox_COCO)
                f.write(f"{category} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

    print("Converting COCO Json to YOLO txt finished!")

In [8]:
high_res_train_list_path = f"{TRAINING_LISTS_PATH}higher_resolution_train.txt"
low_res_train_list_path = f"{TRAINING_LISTS_PATH}lower_resolution_train.txt"
vague_train_list_path = f"{TRAINING_LISTS_PATH}vague_train.txt"
high_res_val_list_path = f"{TRAINING_LISTS_PATH}higher_resolution_val.txt"
low_res_val_list_path = f"{TRAINING_LISTS_PATH}lower_resolution_val.txt"
vague_val_list_path = f"{TRAINING_LISTS_PATH}vague_val.txt"

In [9]:
trains_ids = get_ids_by_train_list(low_res_train_list_path) + get_ids_by_train_list(
    high_res_train_list_path) + get_ids_by_train_list(vague_train_list_path)
val_ids = get_ids_by_train_list(low_res_val_list_path) + get_ids_by_train_list(
    high_res_val_list_path) + get_ids_by_train_list(vague_val_list_path)

In [10]:
len(trains_ids), len(val_ids)

(9202, 3067)

In [11]:
# copy_photos_and_txt(trains_ids, DATASET_PATH, os.path.join(DATASET_PATH, "train"))
# copy_photos_and_txt(val_ids, DATASET_PATH, os.path.join(DATASET_PATH, "val"))

In [12]:
# jpg_files = [f for f in os.listdir(f'{DATASET_PATH}/train/') if f.endswith('.jpg')]
# txt_files = [f for f in os.listdir(f'{DATASET_PATH}/train/') if f.endswith('.txt')]
# 
# missing_txt = []
# for jpg_file in jpg_files:
#     txt_file = jpg_file[:-3] + 'txt'  # Replace .jpg with .txt
#     if txt_file not in txt_files:
#         missing_txt.append(txt_file)
# 
# print("Missing TXT files:", missing_txt)

In [13]:
model = YOLO(MODEL_WEIGHTS)
model.info()

YOLOv9c summary: 618 layers, 25533087 parameters, 0 gradients, 103.7 GFLOPs


(618, 25533087, 0, 103.7001216)

In [14]:
torch.cuda.is_available(), torch.cuda.device_count()

(True, 1)

In [15]:
# convert_coco_json_to_yolo_txt("./DATASET_BIG_YOLO/", "./DATASET_BIG/dataset/annotations.json")

In [16]:
def remove_excluded_categories(directory, excluded_categories):
    """
    Removes lines from YOLO annotation files starting with excluded category IDs.

    Args:
        directory: The directory containing the YOLO TXT files.
        excluded_categories: A list of category IDs to exclude (e.g., [5, 6]).
    """
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r") as f:
                lines = f.readlines()

            filtered_lines = [line for line in lines if not line.startswith(
                tuple(map(str, excluded_categories)))]

            with open(filepath, "w") as f:
                f.writelines(filtered_lines)

In [17]:
# directory = f"{DATASET_PATH}/val"
# excluded_categories = [5, 6]
# 
# remove_excluded_categories(directory, excluded_categories)

In [18]:
model.ckpt

{'epoch': -1,
 'best_fitness': None,
 'model': DetectionModel(
   (model): Sequential(
     (0): Conv(
       (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU(inplace=True)
     )
     (1): Conv(
       (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU(inplace=True)
     )
     (2): RepNCSPELAN4(
       (cv1): Conv(
         (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
         (act): SiLU(inplace=True)
       )
       (cv2): Sequential(
         (0): RepCSP(
           (cv1): Conv(
             (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
     

In [19]:
print(f"Training on GPU? {torch.cuda.is_available()}")
results = model.train(trainer=None, 
                      resume=False, 
                      data=YAML_PATH, 
                      epochs=50, 
                      imgsz=1024, 
                      save_period=10, 
                      cache=True, 
                      batch=-1, 
                      project=OUTPUT_DIR,
                      device=0)

Training on GPU? True
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=./yolov9c-50k_iter-output/train5/weights/best.pt, data=./DATASET_BIG_YOLO/AGAR.yaml, epochs=50, time=None, patience=100, batch=-1, imgsz=1024, save=True, save_period=10, cache=True, device=0, workers=8, project=./yolov9c-50k_iter-output, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning /home/user/DATASET_BIG_YOLO/train.cache... 9202 images, 2 backgrounds, 18 corrupt: 100%|██████████| 9202/9202 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/train/12160.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/train/12269.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/train/12316.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0007]
train: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/train/12377.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0195]
train: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/train/12607.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0059      1.0006]
train: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/train/13672.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/train/3417.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.027]
train: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/train/3440.jpg: ignoring corrupt image/label: non-normalized or out o

train: 36.8GB RAM required to cache images with 50% safety margin but only 21.2/26.7GB available, not caching images ⚠️


val: Scanning /home/user/DATASET_BIG_YOLO/val.cache... 3067 images, 0 backgrounds, 10 corrupt: 100%|██████████| 3067/3067 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/val/11988.jpg: ignoring corrupt image/label: negative label values [   -0.47082]
val: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/val/12586.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0346       1.033      1.0707]
val: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/val/15250.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1492]
val: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/val/5794.jpg: ignoring corrupt image/label: negative label values [   -0.16099]
val: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/val/6360.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0115]
val: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/val/6805.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0064       1.004       1.007]
val: WARNING ⚠️ /home/user/DATASET_BIG_YOLO/val/7093.jpg: ignoring corrupt image/label: non-normalized or out of boun


val: Caching images (8.2GB RAM): 100%|██████████| 3057/3057 [01:12<00:00, 42.11it/s] 


Plotting labels to yolov9c-50k_iter-output/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 154 weight(decay=0.0), 161 weight(decay=0.0005078125), 160 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to yolov9c-50k_iter-output/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      12.2G      1.056     0.4946      1.038        142       1024:  25%|██▍       | 452/1837 [02:07<06:09,  3.75it/s]

: 

In [ ]:
model.val()